## **Data Ingestion and Cleaning**


In [0]:
#Displaying the files and directories for checking whether the dataset is correctly uploaded
display(dbutils.fs.ls("/FileStore/tables/"))

path,name,size,modificationTime
dbfs:/FileStore/tables/Chicago_Crime_Data.csv,Chicago_Crime_Data.csv,1958028100,1745596666000


In [0]:
#Loading the csv file
df = spark.read.csv("/FileStore/tables/Chicago_Crime_Data.csv", header = True, inferSchema = True)

In [0]:
#Displaying the first 5 rows of the dataframe
df.show(5)

+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|13311263|   JG503434|07/29/2022 03:39:...|     023XX S TROY ST|1582|OFFENSE INVOLVING...|   CHILD PORNOGRAPHY|           RE

In [0]:
#Printing the schema
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [0]:
#Counting the total number of rows
row_count = df.count()
print(f"Total Rows: {row_count}")

#Counting the total number of columns
col_count = len(df.columns)
print(f"Total Columns: {col_count}")

Total Rows: 8287103
Total Columns: 22


In [0]:
#Creating a temporary SQL view
df.createOrReplaceTempView("crime_data")

In [0]:
#Counting the rows with a condition of ward=25 in SQL
spark.sql("""select count(*) from crime_data where ward = 25""").display()

count(1)
112551


In [0]:
#Counting the distinct values and displaying the results
spark.sql("""select count(distinct ward) from crime_data""").display()

count(DISTINCT ward)
50


In [0]:
#Counting crimes by primary type and displaying them in a descending order
spark.sql("""select `primary type`, count(*) as total_count from crime_data group by `primary type` order by total_count desc""").display()

primary type,total_count
THEFT,1756670
BATTERY,1509985
CRIMINAL DAMAGE,942617
NARCOTICS,759840
ASSAULT,551605
OTHER OFFENSE,516016
BURGLARY,439783
MOTOR VEHICLE THEFT,421081
DECEPTIVE PRACTICE,378954
ROBBERY,311168


In [0]:
#Importing column operations and sum function
from pyspark.sql.functions import col, sum as _sum

#Removing the rows where Case Number and Primary type is 0
df = df.filter((col("Case Number") != "0") & (col("Primary Type") != "0"))

In [0]:
#Removing the rows where ID is 0
df = df.filter(col("ID") != 0)

In [0]:
#Displaying the first 5 rows of these selected columns
df.select("ID", "Case Number", "Primary Type", "Latitude", "Longitude").show(5)

+--------+-----------+--------------------+------------+-------------+
|      ID|Case Number|        Primary Type|    Latitude|    Longitude|
+--------+-----------+--------------------+------------+-------------+
|13311263|   JG503434|OFFENSE INVOLVING...|        null|         null|
|13053066|   JG103252|           NARCOTICS|        null|         null|
|11227634|   JB147599| CRIM SEXUAL ASSAULT|        null|         null|
|13203321|   JG415333|     CRIMINAL DAMAGE|41.886018055|-87.633937881|
|13204489|   JG416325|               THEFT|41.871834768| -87.62615082|
+--------+-----------+--------------------+------------+-------------+
only showing top 5 rows



In [0]:
#Dropping the null values
df = df.dropna(subset=["Primary Type", "Date", "Latitude", "Longitude"])

#Filling the null values
df = df.fillna({"Location Description": "UNKNOWN"})

In [0]:
#Dropping duplicate columns
df = df.dropDuplicates(["Case Number"])

In [0]:
#Verifying the duplicates
df.groupBy(df.columns).count().filter("count > 1").count()

Out[15]: 0

In [0]:
#Keeping the rows where the years are between 2020 and 2025
df = df.filter((col("Year") >= 2000) & (col("Year") <= 2025))

#Filtering the rows based on Latitude and Longitude
df = df.filter((col("Latitude").between(41.0, 42.0)) & (col("Longitude").between(-88.0, -87.0)))

In [0]:
#Importing the necessary functions
from pyspark.sql.functions import upper, col

#Converting the values in Primary type and Location Description to Uppercase
df = df.withColumn("Primary Type", upper(col("Primary Type")))
df = df.withColumn("Location Description", upper(col("Location Description")))

In [0]:
#importing the time stamp function
from pyspark.sql.functions import to_timestamp

#Converting the Date string into timestamp format
df = df.withColumn("Date", to_timestamp("Date", "MM/dd/yyyy hh:mm:ss a"))

In [0]:
#Printing the updated schema
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = false)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [0]:
#Counting the null values
null_values = df.select([
    _sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])

#Displaying the null values
null_values.show()

+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+
| ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|  Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|  0|          0|   0|    0|   0|           0|          0|                   0|     0|       0|   0|      46|592226|        590939|       0|           0|           0|   0|         0|       0|        0|       0|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+---

In [0]:
#Counting the rows where ID, Primary Type and Latitude are 0
df.filter(col("ID") == 0).count()
df.filter(col("Primary Type") == "0").count()
df.filter(col("Latitude") > 90).count()

Out[21]: 0

In [0]:
#Counting the distinct case number values
df.select("Case Number").distinct().count()

Out[22]: 8025603

In [0]:
#Deleting the location column
df = df.drop("Location")

In [0]:
#Verifying the nulls using the ID from crime_data with a condition
spark.sql("""
select id from crime_data
where district is null""").display()

id
3575885
3596991
4437079
4740376
6376239
6580042
6549802
6585169
6577661
6580306


In [0]:
#Filling the null values with -1
df = df.fillna({"Ward": -1, "Community Area": -1})

In [0]:
#Keeping the rows where the values are not Null in District column
df = df.filter(col("District").isNotNull())

In [0]:
#Counting the null values
null_values = df.select([
    _sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])

#Displaying the null values
null_values.show()

+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+
| ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+
|  0|          0|   0|    0|   0|           0|          0|                   0|     0|       0|   0|       0|   0|             0|       0|           0|           0|   0|         0|       0|        0|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+


In [0]:
#Counting the total rows
df.count()


Out[28]: 8025557

## **Exploratory Data Analysis**

Summary Statistics using describe()

In [0]:
# Displaying summary statistics (count, mean, stddev, min, max)
df.describe().show()


+-------+------------------+------------------+------------------+------------------+-----------------+---------------+--------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+--------------------+------------------+-------------------+
|summary|                ID|       Case Number|             Block|              IUCR|     Primary Type|    Description|Location Description|              Beat|          District|              Ward|   Community Area|          FBI Code|      X Coordinate|      Y Coordinate|              Year|          Updated On|          Latitude|          Longitude|
+-------+------------------+------------------+------------------+------------------+-----------------+---------------+--------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-----

In [0]:
# Displaying extended summary statistics (count, mean, stddev, ,in, 25%, 50%, 75%, max) for selected columns
df.select(
    "Year", "Beat", "District", "Ward", 
    "Community Area", "Latitude", "Longitude"
).summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").show()

+-------+------------------+------------------+------------------+------------------+-----------------+-------------------+--------------------+
|summary|              Year|              Beat|          District|              Ward|   Community Area|           Latitude|           Longitude|
+-------+------------------+------------------+------------------+------------------+-----------------+-------------------+--------------------+
|  count|           8025557|           8025557|           8025557|           8025557|          8025557|            8025557|             8025557|
|   mean|2010.7710075450216|1158.9437619096095|11.038569659401833|20.523275331543967|35.27701479162132|  41.83902621144674|   -87.6710268200504|
| stddev|  6.89140475302255| 688.7113361198233|  6.79654022986386|14.335212800374576|22.76822889194544|0.08377618747524357|0.059436069557766716|
|    min|              2001|               111|                 1|                -1|               -1|       41.644585429|       

In [0]:
#Creating a temporary SQL view of the cleaned dataframe
df.createOrReplaceTempView("clean_crime_data")

In [0]:
# Preview of the first 10 rows to verify the data for further EDA
spark.sql("""
select * from clean_crime_data
limit 10""").display()

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude
21408,A417216,2010-09-06T07:00:00.000+0000,039XX S VINCENNES AVE,0110,HOMICIDE,FIRST DEGREE MURDER,AUTO,true,false,214,2,4,38,01A,1180666,1879108,2010,09/19/2022 03:41:05 PM,41.823529117,-87.612707977
1900,C153276,2002-12-05T13:50:00.000+0000,055XX W RICE ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,true,false,1524,15,37,25,01A,1139235,1905124,2002,09/19/2022 03:41:05 PM,41.895773675,-87.764072031
1956,D212672,2002-12-31T09:30:00.000+0000,061XX S DREXEL AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,true,false,313,3,20,42,01A,1183061,1864566,2002,09/19/2022 03:41:05 PM,41.783569249,-87.604374166
1375032,F072578,2001-02-05T16:02:00.000+0000,083XX S BAKER AV,0460,BATTERY,SIMPLE,STREET,false,false,424,4,-1,-1,08B,1198542,1849538,2001,08/17/2015 03:03:40 PM,41.74195729,-87.548119531
1391053,F104558,2001-02-20T23:15:00.000+0000,031XX S SACRAMENTO AV,0460,BATTERY,SIMPLE,ALLEY,false,false,1033,10,-1,-1,08B,1156817,1883913,2001,08/17/2015 03:03:40 PM,41.837230208,-87.700071474
1348412,G 050552,2001-01-12T10:00:00.000+0000,010XX N MASSASOIT AV,0820,THEFT,$500 AND UNDER,CHA PARKING LOT/GROUNDS,false,false,1511,15,-1,-1,06,1137793,1906326,2001,08/17/2015 03:03:40 PM,41.899098245,-87.76933924
1309881,G000036,2001-01-01T00:07:00.000+0000,038XX W 65 ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,false,false,833,8,-1,-1,14,1152000,1861189,2001,08/17/2015 03:03:40 PM,41.774968402,-87.718343871
1312205,G000049,2001-01-01T00:15:00.000+0000,113XX S WENTWORTH AV,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,false,false,522,5,-1,-1,14,1176932,1829447,2001,08/17/2015 03:03:40 PM,41.687338661,-87.627900413
1319965,G000066,2001-01-01T00:50:00.000+0000,026XX W 84 PL,1310,CRIMINAL DAMAGE,TO PROPERTY,FOREST PRESERVE,false,false,835,8,-1,-1,14,1160288,1848529,2001,08/17/2015 03:03:40 PM,41.740060801,-87.688308961
1312546,G000145,2001-01-01T01:15:00.000+0000,060XX S MICHIGAN AV,031A,ROBBERY,ARMED: HANDGUN,OTHER,false,false,311,3,-1,-1,03,1178145,1865224,2001,08/17/2015 03:03:40 PM,41.785487819,-87.622377839


In [0]:
# Querying to count the number of crimes per year
spark.sql("""select year, count(*) as total_crimes from clean_crime_data
    group by Year
    order by Year
""").display()

year,total_crimes
2001,472236
2002,461302
2003,462355
2004,457397
2005,440144
2006,436252
2007,426813
2008,411216
2009,378163
2010,362534


Databricks visualization. Run in Databricks to view.

The line graph shows a consistent decline in crime rates over the two decades


In [0]:
# Querying to count the total number of crimes for each Primary Type
spark.sql("""select `Primary Type`, count(*) as total_crimes from clean_crime_data
group by `Primary Type`
order by total_crimes DESC""").display()

Primary Type,total_crimes
THEFT,1696629
BATTERY,1473933
CRIMINAL DAMAGE,915374
NARCOTICS,735298
ASSAULT,539159
OTHER OFFENSE,499244
BURGLARY,428862
MOTOR VEHICLE THEFT,411062
DECEPTIVE PRACTICE,349250
ROBBERY,303387


Databricks visualization. Run in Databricks to view.

The word cloud reveals that 'Theft', 'Battery', and 'Criminal Damage' are the most common crime types in the dataset, as indicated by their larger text sizes. This visualization highlights that a few key crime categories dominate the overall crime landscape.


In [0]:
# Querying to count the total number of crimes in each district and sorting them
spark.sql("""select distinct(district), count(*) as total_crimes from clean_crime_data
group by district
order by total_crimes desc""").display()

district,total_crimes
8,551218
11,519635
6,479993
7,466858
4,465178
25,464183
3,415995
12,411020
9,398612
2,389250


Databricks visualization. Run in Databricks to view.

The bar chart shows that Districts 8 and 11 have the highest total crime counts, each exceeding 500,000 reported incidents. Districts 7 and 25 also report very high crime levels, above 450,000 crimes. In contrast, District 31 has an exceptionally low crime count, suggesting either a very low population or incomplete data. Most other districts fall in the 300,000 to 400,000 range, indicating moderate crime activity across the city.

In [0]:
# SparkSQL Query to count the total number of crimes and calculate the average arrest rate according to each Primary Type of crime
spark.sql("""select distinct(`Primary Type`), count(*) as total_crimes, round(avg(cast(Arrest as int)),2) As Arrest_percentage 
from clean_crime_data
group by `Primary Type`""").display()

Primary Type,total_crimes,Arrest_percentage
OFFENSE INVOLVING CHILDREN,53998,0.19
CRIMINAL SEXUAL ASSAULT,9025,0.07
STALKING,5562,0.13
PUBLIC PEACE VIOLATION,53070,0.63
OBSCENITY,840,0.76
ARSON,13913,0.12
GAMBLING,14361,0.99
CRIMINAL TRESPASS,217243,0.69
ASSAULT,539159,0.2
NON - CRIMINAL,38,0.16


Databricks visualization. Run in Databricks to view.

The plot shows that serious crimes such as Homicide, Kidnapping, and Sex Offense tend to have higher arrest percentages, often exceeding 60%. In contrast, common crimes like Theft, Battery, and Criminal Damage exhibit much lower arrest rates, generally between 10% and 30%, despite their high frequency. This suggests that while serious crimes receive focused enforcement efforts, frequent crimes may face challenges in detection and prosecution. The color intensity highlights the total number of crimes, with Theft appearing as the most common but having a relatively lower arrest rate.

In [0]:
# SparkSQL query to count the crimes by locations and filtering them if the crime count is more than 150,000
spark.sql("""
select `Location Description`, count(*) as total_crimes_in_each_location from clean_crime_data
group by `Location Description`
having total_crimes_in_each_location > 150000
order by total_crimes_in_each_location desc""").display()

Location Description,total_crimes_in_each_location
STREET,2109114
RESIDENCE,1318355
APARTMENT,931494
SIDEWALK,735829
OTHER,258822
PARKING LOT/GARAGE(NON.RESID.),197812
ALLEY,178364
SMALL RETAIL STORE,160117


Databricks visualization. Run in Databricks to view.

The donut chart shows that the majority of crimes occur on streets (35.8%), followed by residences (22.4%), apartments (15.8%), and sidewalks (12.5%). Smaller proportions of crimes happen in areas like parking lots, alleys, and small retail stores. Public spaces and residential areas are the primary hotspots for criminal activity.


In [0]:
# SparkSQL query to count the number of crimes based on whether an arrest was made or not
spark.sql("""
select arrest, count(*) from clean_crime_data
group by arrest""").display()

arrest,count(1)
true,2045465
false,5980092


Databricks visualization. Run in Databricks to view.

The donut chart shows that only 25.5% of crimes led to an arrest, while 74.5% of crimes did not result in an arrest. This suggests that a large proportion of criminal incidents remain unresolved, highlighting challenges in crime detection and law enforcement effectiveness

In [0]:
# SparkSQL query to count the number of crimes based on whether the incident was domestic-related or not
spark.sql("""
select domestic, count(*) from clean_crime_data
group by domestic""").display()

domestic,count(1)
true,1389844
false,6635713


Databricks visualization. Run in Databricks to view.

The donut chart shows that 82.7% of crimes are non-domestic, while 17.3% are domestic-related incidents. This indicates that the majority of crimes reported occur outside of domestic contexts, though domestic incidents still represent a significant share of overall crime.

## **Feature** **Engineering**

In [0]:
# Importing necessary packages for feature analysis
from pyspark.sql.functions import hour, dayofweek, month, col
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.stat import Correlation, ChiSquareTest

# Adding time-based features to a new DataFrame by segmenting the Date column
df_model = df.withColumn("Hour", hour("Date")) \
             .withColumn("DayOfWeek", dayofweek("Date")) \
             .withColumn("Month", month("Date"))

In [0]:
# Selecting only the necessary columns for the ML model
selected_cols = ["Primary Type", "Location Description", "Community Area", "Ward", "Hour", "DayOfWeek", "Month", "Domestic"]
df_model = df_model.select(*selected_cols)

In [0]:
# Creating numerical indexes for categorical columns to prepare the data for the ML model 
index_columns = ["Primary Type", "Location Description", "Community Area", "Ward"]
for c_name in index_columns:
    index = StringIndexer(inputCol=c_name, outputCol=c_name.replace(" ", "") + "_index")
    df_model = index.fit(df_model).transform(df_model)


In [0]:
# Converting 'Domestic' column from boolean to string and labelling it to use it as target variable for the ML model
df_model = df_model.withColumn("Domestic_str", col("Domestic").cast("string"))
target_index = StringIndexer(inputCol="Domestic_str", outputCol="label")
df_model = target_index.fit(df_model).transform(df_model)


In [0]:
# Using VectorAssembler to group all the required columns in one single vector for calculating the correlation
corr_cols = ["Hour", "DayOfWeek", "Month"]
corr_assemble = VectorAssembler(inputCols=corr_cols, outputCol="features_corr")
corr_df = corr_assemble.transform(df_model)


In [0]:
# Finding correlation matrix between numerical input columns using .corr() method
correlation = Correlation.corr(corr_df, "features_corr", "pearson").head()[0].toArray()
print("Correlation Matrix :\n", correlation)

Correlation Matrix :
 [[ 1.00000000e+00  1.55647099e-02 -3.48408119e-04]
 [ 1.55647099e-02  1.00000000e+00 -2.57195026e-03]
 [-3.48408119e-04 -2.57195026e-03  1.00000000e+00]]


In [0]:
# Using chi-square test to check the dependency of categorical input columns
chi_features = [col.replace(" ", "") + "_index" for col in index_columns]
chi_assembler = VectorAssembler(inputCols=chi_features, outputCol="features_chi")
chi_df = chi_assembler.transform(df_model)
chi_result = ChiSquareTest.test(chi_df, "features_chi", "label").head()

print("Chi-Square Test Results:")
print("p-values:", chi_result.pValues)
print("statistics:", chi_result.statistics)
print("degrees of freedom:", chi_result.degreesOfFreedom)

Chi-Square Test Results:
p-values: [0.0,0.0,0.0,0.0]
statistics: [2375594.0839359504,1483082.7807249394,190255.6078657329,177718.9112082372]
degrees of freedom: [35, 217, 78, 50]


In [0]:
# Creating a temporary view of the original DataFrame
df.createOrReplaceTempView("check")

In [0]:
# SparkSQL query for verifying the original DataFrame
spark.sql("""select * from check limit 5""").display()

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude
21408,A417216,2010-09-06T07:00:00.000+0000,039XX S VINCENNES AVE,0110,HOMICIDE,FIRST DEGREE MURDER,AUTO,true,false,214,2,4,38,01A,1180666,1879108,2010,09/19/2022 03:41:05 PM,41.823529117,-87.612707977
1900,C153276,2002-12-05T13:50:00.000+0000,055XX W RICE ST,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,true,false,1524,15,37,25,01A,1139235,1905124,2002,09/19/2022 03:41:05 PM,41.895773675,-87.764072031
1956,D212672,2002-12-31T09:30:00.000+0000,061XX S DREXEL AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,true,false,313,3,20,42,01A,1183061,1864566,2002,09/19/2022 03:41:05 PM,41.783569249,-87.604374166
1375032,F072578,2001-02-05T16:02:00.000+0000,083XX S BAKER AV,0460,BATTERY,SIMPLE,STREET,false,false,424,4,-1,-1,08B,1198542,1849538,2001,08/17/2015 03:03:40 PM,41.74195729,-87.548119531
1391053,F104558,2001-02-20T23:15:00.000+0000,031XX S SACRAMENTO AV,0460,BATTERY,SIMPLE,ALLEY,false,false,1033,10,-1,-1,08B,1156817,1883913,2001,08/17/2015 03:03:40 PM,41.837230208,-87.700071474


In [0]:
# Creating a temporary view of the dataframe [df_model], which has undergone feature transformation
df_model.createOrReplaceTempView("modeldf_check")

In [0]:
# SparkSQL query for verifying the feature transformed DataFrame
spark.sql("""
select * from modeldf_check
limit 5""").display()

Primary Type,Location Description,Community Area,Ward,Hour,DayOfWeek,Month,Domestic,PrimaryType_index,LocationDescription_index,CommunityArea_index,Ward_index,Domestic_str,label
HOMICIDE,AUTO,38,4,7,2,9,false,21.0,90.0,28.0,20.0,false,0.0
HOMICIDE,STREET,25,37,13,5,12,false,21.0,0.0,1.0,15.0,false,0.0
HOMICIDE,STREET,42,20,9,3,12,false,21.0,0.0,25.0,8.0,false,0.0
BATTERY,STREET,-1,-1,16,2,2,false,1.0,0.0,0.0,0.0,false,0.0
BATTERY,ALLEY,-1,-1,23,3,2,false,1.0,6.0,0.0,0.0,false,0.0


## ML MODEL IMPLEMENTATION

In [0]:
# Defining a list of column names to give these columns as inputs for the ML model
input_columns = ["PrimaryType_index", "LocationDescription_index", "CommunityArea_index", "Ward_index", "Hour", "DayOfWeek", "Month"]

# Used VectorAssembler to combine all individual input columns into one single column 
v_assembler = VectorAssembler(inputCols=input_columns, outputCol="features")
df_model = v_assembler.transform(df_model)
df_model.cache()

Out[17]: DataFrame[Primary Type: string, Location Description: string, Community Area: int, Ward: int, Hour: int, DayOfWeek: int, Month: int, Domestic: boolean, PrimaryType_index: double, LocationDescription_index: double, CommunityArea_index: double, Ward_index: double, Domestic_str: string, label: double, features: vector]

In [0]:
# Importing necessary packages to implement Random Forest algorithm
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

# Spliting the dataframe into training and testing set
df_model = df_model.repartition(10)
training_df, testing_df = df_model.randomSplit([0.8, 0.2], seed=42)

In [0]:
# Creating the Random Forest model by specifying the appropriate hyperparameters
random_forest = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=20,
    maxDepth=7,
    maxBins=300
)
# Training the model by giving the training set as input
model = random_forest.fit(training_df)

In [0]:
# Creating a variable to predict the model's performance
predictions = model.transform(testing_df)

In [0]:
# Importing necessary packages to evaluate the model's performance
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluating the model's performance using multiple metrics
accuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")


print(f"Accuracy:", accuracy.evaluate(predictions))
print("F1 Score:", evaluator_f1.evaluate(predictions))
print("Precision:", evaluator_precision.evaluate(predictions))
print("Recall:", evaluator_recall.evaluate(predictions))

Accuracy: 0.8875693159949637
F1 Score: 0.8780718680565275
Precision: 0.8794603302643548
Recall: 0.8875693159949638
